# BCS Testing

In [1]:
import numpy as np
from scipy.stats import qmc

import PyUQTk.uqtkarray as uqtkarray
import PyUQTk.pce as uqtkpce
import PyUQTk.PyPCE.pce_tools as pce_tools

import bcs as bcs
import math

import pandas as pd

PyMC is required for some of the MCMC postprocessing codes.
Will proceed without, but some convergence tests will not be available.


## Inputs

In [2]:
nord = 4            # order of the polynomial
ndim = 3            # number of dimensions
pc_type="LU"        # type of PC

c_opt = "alt"        # method for selecting starting coefficients

pc_alpha = 0.0      # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
pc_beta = 1.0       # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs

## Initialize PC

In [3]:
pc_start = uqtkpce.PCSet("NISPnoq", 2, ndim, pc_type, pc_alpha, pc_beta)
pc_model = uqtkpce.PCSet("NISPnoq", nord, ndim, pc_type, pc_alpha, pc_beta)

## BSC Inputs

In [4]:
sigma = np.array([1.e-08]) # inital noise variance; updated in BCS
eta = 1e-15                # algorithm stops once evidence gain falls below this threshold
#lambda_init = np.array([]) # paramter of the laplace distribution; leave empty or set lambda to a fixed nonnegative value

## Define coefficients

In [5]:
npce=pc_model.GetNumberPCTerms()

# sequence of increasing integers
if (c_opt=="seq"):
    coef=np.array([1/(i+1) for i in range(npce)])
    
# sequence of increasing integers alternating with zero    
elif (c_opt=="alt"):
    coef=np.zeros(npce)
    for i in range(npce):
        if (i%2==0):
            coef[i]=1/(i+1)

## Training points

In [6]:
npce=pc_model.GetNumberPCTerms()
nTest=int(npce*2)
rng = qmc.LatinHypercube(d=ndim, seed=42)

x_tr=2*rng.random(n=nTest)-1 # [-1,1]
y_tr=pce_tools.UQTkEvaluatePCE(pc_model, coef, x_tr)

## Validation points

In [7]:
x_val=2*rng.random(n=nTest)-1
y_val=pce_tools.UQTkEvaluatePCE(pc_model, coef, x_val)

## Pick Best Eta

## Testing Points

In [8]:
x_test=2*rng.random(n=nTest)-1 # [-1,1]
y_test=pce_tools.UQTkEvaluatePCE(pc_model, coef, x_test)

## Calculate Errors

In [9]:
c_k, pc_model_final = pce_tools.UQTkBCS(pc_model, y_tr, x_tr, sigma, eta, upit=2)

Starting with 35
18  nonzero basis terms selected out of 35
Multiindex resized from 18 to 26.
Iter. 1 order 3 , 18  nonzero basis terms out of 26
Multiindex resized from 18 to 26.
Iter. 2 order 4 , 18  nonzero basis terms out of 26


In [10]:
if (c_k.shape==coef.shape):
    MSE = np.square(np.subtract(c_k, coef)).mean()
    RMSE=math.sqrt(MSE)
    print(RMSE/np.linalg.norm(y_test))

In [11]:
pce_evals=pce_tools.UQTkEvaluatePCE(pc_model_final, c_k, x_test)
MSE = np.square(np.subtract(y_test, pce_evals)).mean()
RMSE=math.sqrt(MSE)
RMSE/np.linalg.norm(y_test)

1.8923543556251187e-16

In [12]:
print("Nonzero coefficients found:  ", np.nonzero(c_k)[0].shape[0])
print("Total nonzero coefficients:  ", np.nonzero(coef)[0].shape[0])

Nonzero coefficients found:   18
Total nonzero coefficients:   18


In [13]:
if (c_k.shape==coef.shape):
    pd.DataFrame(data=np.transpose([c_k, coef]), columns=['bcs', 'actual'])

In [14]:
# e_tr=1/np.power(10,[i for i in range(1,13)])
# E_v=[]
# for j, eta in enumerate(e_tr):
#     c_tr=pce_tools.UQTkBCS(pc_model, y_tr, x_tr, sigma, eta)
#     phi_v_c_tr=pce_tools.UQTkEvaluatePCE(pc_model, c_tr, x_val)
#     E_v.append(math.sqrt(np.square(np.subtract(y_val, phi_v_c_tr)).mean()))
# eta_opt = e_tr[E_v.index(np.min(E_v))]
# eta_opt